In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/2018-new/2018-02-btc-krw.csv")

In [3]:
df.dtypes

timestamp     object
quantity     float64
price          int64
fee          float64
amount         int64
side           int64
dtype: object

In [4]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price                 int64
fee                 float64
amount                int64
side                  int64
dtype: object

In [5]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [6]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2018-02-14 00:11:39,0.7990,9861000,0.0,-7878939,0,2018-02-14,2018,2,February,14,00:11:39,0,11,39
1,2018-02-14 00:12:12,0.7990,9854000,0.0,7873346,1,2018-02-14,2018,2,February,14,00:12:12,0,12,12
2,2018-02-14 00:15:04,0.1050,9849000,0.0,-1034145,0,2018-02-14,2018,2,February,14,00:15:04,0,15,4
3,2018-02-14 00:15:41,0.1050,9860000,0.0,1035300,1,2018-02-14,2018,2,February,14,00:15:41,0,15,41
4,2018-02-14 00:20:27,1.5000,9859000,0.0,-14788500,0,2018-02-14,2018,2,February,14,00:20:27,0,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18362,2018-02-28 23:59:43,0.0100,11924000,0.0,119240,1,2018-02-28,2018,2,February,28,23:59:43,23,59,43
18363,2018-02-28 23:59:43,0.0002,11924000,0.0,2385,1,2018-02-28,2018,2,February,28,23:59:43,23,59,43
18364,2018-02-28 23:59:48,0.0296,11923000,0.0,352921,1,2018-02-28,2018,2,February,28,23:59:48,23,59,48
18365,2018-02-28 23:59:49,0.5662,11923000,0.0,6750803,1,2018-02-28,2018,2,February,28,23:59:49,23,59,49


In [7]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [8]:
group_day_amount.size()

timestamp_year_month_day
2018-02-14     931
2018-02-15    1487
2018-02-16    1208
2018-02-17     688
2018-02-18    1047
2018-02-19     892
2018-02-20    1523
2018-02-21    2212
2018-02-22    2119
2018-02-23    1373
2018-02-24     915
2018-02-25     689
2018-02-26    1148
2018-02-27    1058
2018-02-28    1077
Name: amount, dtype: int64

In [9]:
group_day_amount

In [10]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2018-02-14   -13986080
2018-02-15      734092
2018-02-16    17687643
2018-02-17     1432140
2018-02-18   -12068675
2018-02-19    13935899
2018-02-20     1998820
2018-02-21     -274696
2018-02-22    -7300157
2018-02-23    11705392
2018-02-24     1013740
2018-02-25      570258
2018-02-26   -10570854
2018-02-27    -6746483
2018-02-28    21926406
Name: amount, dtype: int64

In [11]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [12]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2018-02-14     587
2018-02-15     867
2018-02-16     684
2018-02-17     357
2018-02-18     560
2018-02-19     517
2018-02-20     908
2018-02-21    1293
2018-02-22    1235
2018-02-23     842
2018-02-24     581
2018-02-25     426
2018-02-26     684
2018-02-27     583
2018-02-28     633
Name: amount, dtype: int64

In [13]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2018-02-14    1531487209
2018-02-15    2529253711
2018-02-16    2228838934
2018-02-17    1374558420
2018-02-18    2165526961
2018-02-19    1759974379
2018-02-20    3007532623
2018-02-21    5373720824
2018-02-22    4978452013
2018-02-23    3178734522
2018-02-24    1965575540
2018-02-25    1612991345
2018-02-26    2736765547
2018-02-27    2866889083
2018-02-28    2507614791
Name: amount, dtype: int64

In [14]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [15]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2018-02-14    344
2018-02-15    620
2018-02-16    524
2018-02-17    331
2018-02-18    487
2018-02-19    375
2018-02-20    615
2018-02-21    919
2018-02-22    884
2018-02-23    531
2018-02-24    334
2018-02-25    263
2018-02-26    464
2018-02-27    475
2018-02-28    444
Name: amount, dtype: int64

In [16]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2018-02-14   -1545473289
2018-02-15   -2528519619
2018-02-16   -2211151291
2018-02-17   -1373126280
2018-02-18   -2177595636
2018-02-19   -1746038480
2018-02-20   -3005533803
2018-02-21   -5373995520
2018-02-22   -4985752170
2018-02-23   -3167029130
2018-02-24   -1964561800
2018-02-25   -1612421087
2018-02-26   -2747336401
2018-02-27   -2873635566
2018-02-28   -2485688385
Name: amount, dtype: int64

In [17]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2018-02-14     3076960498
2018-02-15     5057773330
2018-02-16     4439990225
2018-02-17     2747684700
2018-02-18     4343122597
2018-02-19     3506012859
2018-02-20     6013066426
2018-02-21    10747716344
2018-02-22     9964204183
2018-02-23     6345763652
2018-02-24     3930137340
2018-02-25     3225412432
2018-02-26     5484101948
2018-02-27     5740524649
2018-02-28     4993303176
Name: amount, dtype: int64

In [18]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2018-02-14     931
 2018-02-15    1487
 2018-02-16    1208
 2018-02-17     688
 2018-02-18    1047
 2018-02-19     892
 2018-02-20    1523
 2018-02-21    2212
 2018-02-22    2119
 2018-02-23    1373
 2018-02-24     915
 2018-02-25     689
 2018-02-26    1148
 2018-02-27    1058
 2018-02-28    1077
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-02-14   -13986080
 2018-02-15      734092
 2018-02-16    17687643
 2018-02-17     1432140
 2018-02-18   -12068675
 2018-02-19    13935899
 2018-02-20     1998820
 2018-02-21     -274696
 2018-02-22    -7300157
 2018-02-23    11705392
 2018-02-24     1013740
 2018-02-25      570258
 2018-02-26   -10570854
 2018-02-27    -6746483
 2018-02-28    21926406
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-02-14     587
 2018-02-15     867
 2018-02-16     684
 2018-02-17     357
 2018-02-18     560
 2018-02-19     517
 2018-02-20     908
 2018-02-21    1293
 2018-02-22    1235
 2018-02-23     842
 201

In [19]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-02-14,931,-13986080,587,1531487209,344,-1545473289,3076960498,1545473289
2018-02-15,1487,734092,867,2529253711,620,-2528519619,5057773330,2528519619
2018-02-16,1208,17687643,684,2228838934,524,-2211151291,4439990225,2211151291
2018-02-17,688,1432140,357,1374558420,331,-1373126280,2747684700,1373126280
2018-02-18,1047,-12068675,560,2165526961,487,-2177595636,4343122597,2177595636
2018-02-19,892,13935899,517,1759974379,375,-1746038480,3506012859,1746038480
2018-02-20,1523,1998820,908,3007532623,615,-3005533803,6013066426,3005533803
2018-02-21,2212,-274696,1293,5373720824,919,-5373995520,10747716344,5373995520
2018-02-22,2119,-7300157,1235,4978452013,884,-4985752170,9964204183,4985752170


In [20]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2018-02-14                              931     -13986080   
2018-02-15                             1487        734092   
2018-02-16                             1208      17687643   
2018-02-17                              688       1432140   
2018-02-18                             1047     -12068675   
2018-02-19                              892      13935899   
2018-02-20                             1523       1998820   
2018-02-21                             2212       -274696   
2018-02-22                             2119      -7300157   
2018-02-23                             1373      11705392   
2018-02-24                              915       1013740   
2018-02-25                              689        570258   
2018-02-26                             1148     -10570854   
2018-02-27                             1058      -6746483   
2018-02-28              

In [21]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/2018-02-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [22]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/2018-02-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-02-14,931,-13986080,587,1531487209,344,-1545473289,3076960498,1545473289
2018-02-15,1487,734092,867,2529253711,620,-2528519619,5057773330,2528519619
2018-02-16,1208,17687643,684,2228838934,524,-2211151291,4439990225,2211151291
2018-02-17,688,1432140,357,1374558420,331,-1373126280,2747684700,1373126280
2018-02-18,1047,-12068675,560,2165526961,487,-2177595636,4343122597,2177595636
2018-02-19,892,13935899,517,1759974379,375,-1746038480,3506012859,1746038480
2018-02-20,1523,1998820,908,3007532623,615,-3005533803,6013066426,3005533803
2018-02-21,2212,-274696,1293,5373720824,919,-5373995520,10747716344,5373995520
2018-02-22,2119,-7300157,1235,4978452013,884,-4985752170,9964204183,4985752170
